# Impl

In [14]:
#| default_exp impl

In [15]:
#|export
from dataclasses import dataclass
from typing import Any,Union,Tuple,List

from fastcore.utils import *
from fastcore.xml import *
from fastcore.xtras import hl_md
from fasthtml import *

In [16]:
import httpx
from pprint import pprint
from IPython.display import HTML,Markdown

In [17]:
@patch
def _repr_html_(self:httpx.Response): return self.text

def md(self): return Markdown(f'```xml\n{self.text}```')

In [18]:
show(picocondlink)

In [19]:
set_pico_cls()

<IPython.core.display.Javascript object>

In [20]:
#|export
id_curr = 'current-todo'
id_list = 'todo-list'
def tid(id): return f'todo-{id}'

In [21]:
#|export
@dataclass
class TodoItem():
    title: str; id: int = -1; done: bool = False

    def __xt__(self):
        show = AX(self.title, f'/todos/{self.id}', id_curr)
        edit = AX('edit',     f'/edit/{self.id}' , id_curr)
        dt = ' (done)' if self.done else ''
        return Li(show, dt, ' | ', edit, id=tid(self.id))

    _repr_html_ = showtags

#|export
TODO_LIST = [TodoItem(id=0, title="Start writing todo list", done=True),
             TodoItem(id=1, title="???", done=False),
             TodoItem(id=2, title="Profit", done=False)]

In [22]:
todo = TODO_LIST[0]
todo

TodoItem(title='Start writing todo list', id=0, done=True)

In [23]:
show(todo)

In [24]:
show(Ul(*TODO_LIST))

In [25]:
#| export
def mk_input(**kw): return Input(id="new-title", name="title", placeholder="New Todo", **kw)

def get_card(todos):
    add = Form(Group(mk_input(), Button("Add")),
               hx_post="/", target_id=id_list, hx_swap="beforeend")
    return Card(Ul(*todos, id=id_list),
                header=add, footer=Div(id=id_curr))

In [26]:
show(get_card(TODO_LIST))

In [12]:
#| export
def find_todo(id):
    try: return next(o for o in TODO_LIST if o.id==id)
    except: raise NotFoundException(f'Todo #{id}') from None

In [13]:
id = 2
todo = find_todo(id)
print(todo)

TodoItem(title='Profit', id=2, done=False)


In [14]:
#| export
def get_editform(id):
    todo = find_todo(id)
    res = Form(Group(Input(id="title"), Button("Save")),
        Hidden(id="id"), Checkbox(id="done", label='Done'),
        hx_put="/", target_id=tid(id), id="edit")
    fill_form(res, todo)
    return res

In [15]:
edit = get_editform(2)
show(edit)

In [16]:
find_elems(edit, 'input', 'value', id='title')

['Profit']

## Testing

In [17]:
r = httpx.get('http://localhost:8000')

In [19]:
# md(r)

## export -

In [27]:
#| eval: false
#| hide
from nbdev.export import nb_export
nb_export('impl.ipynb', '.')